### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6681,2023-11-14,Brasil Nbb,19:30,Pato,Bauru,2.69,1.45,151.5,1.88,1.79,4.5,1.94,1.77
6682,2023-11-14,Brasil Nbb,20:00,Corinthians Paulista,São Paulo,1.76,2.02,158.5,1.87,1.79,-2.5,2.01,1.72
6683,2023-11-14,Eua Nba,00:00,Sacramento Kings,Cleveland Cavaliers,2.01,1.93,225.5,1.86,2.07,1.5,1.88,2.07
6684,2023-11-14,Eua Nba,21:00,Charlotte Hornets,Miami Heat,2.21,1.56,226.5,1.87,1.79,1.5,2.03,1.66
6685,2023-11-14,Eua Nba,21:00,Detroit Pistons,Atlanta Hawks,2.78,1.45,232.5,1.88,1.96,5.5,1.90,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6775,2023-11-14,México Lnbp,23:00,Halcones de Xalapa,Fuerza Regia,2.81,1.42,158.5,1.82,1.92,4.5,1.99,1.75
6776,2023-11-14,México Lnbp,23:00,Plateros,Astros,2.18,1.66,176.5,1.86,1.88,1.5,2.02,1.71
6777,2023-11-14,México Lnbp,23:00,Santos,Abejas,2.03,1.75,167.5,1.80,1.93,1.5,1.89,1.85
6778,2023-11-14,México Lnbp,23:30,Panteras,Libertadores,1.77,2.01,171.5,1.87,1.87,-2.5,2.00,1.74


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Nba,Sacramento Kings,Cleveland Cavaliers,225.5,1.86,1.0000,Over
4,21:00,Eua Nba,Detroit Pistons,Atlanta Hawks,232.5,1.88,0.7044,Over
